In [ ]:
import numpy as np

import sys
sys.path.insert(0, '..')

from paus_utils import plot_PAUS_source, w_central

import matplotlib.pyplot as plt

from jpasLAEs.utils import mag_to_flux

In [ ]:
directory = "/home/alberto/almacen/PAUS_data/20240207_mock_data_with_FluxLines_columns_MR_150vols_3x3deg_z0-5"  # path where the data is store
suff_name = "_magCut[PAUS_BBF_i_25]_LC_chunks[0-150].npy"  # all the names of single 'columns' arrays end with this string

columns_to_read = ["Pos", "Redshift", "ObsMagDust", 'ObsAppMagDustLine',
                   'FluxDust_OII_3727', 'FluxDust_OII_3729']

OUT = {'dummy' : 'dummy_string'}
for nm in columns_to_read:
    fname = f'{directory}/{nm}{suff_name}'
    OUT[nm] = np.load( fname )
OUT.pop('dummy')

In [ ]:
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u


src = np.random.randint(0, len(OUT['Redshift']))

def M_to_m(M, redshift):
    '''
    Computes the apparent magnitude from the absolute magnitude
    Inputs:
    M: Absolute magnitude
    redshift: Redshift of the source
    '''
    # Luminosity distance:
    dL = cosmo.luminosity_distance(redshift).to(u.pc).value.reshape(-1, 1)

    return M + 5 * np.log10(dL) - 5

filter_order = np.concatenate([np.arange(1, 3),
                               np.arange(4, 21),
                               np.arange(22, 35),
                               np.arange(36, 44),
                               [0, 3, 21, 35, 44, 45]])

flx = mag_to_flux(M_to_m(OUT['ObsAppMagDustLine'].T[:, src][filter_order], OUT['Redshift'][src]), w_central).flatten()
print(flx)

fig, ax = plt.subplots()

plot_PAUS_source(flx, flx * 0.01,
                 ax=ax)

plt.show()

In [ ]:
def M_to_m_alt(M, redshift):
    '''
    Computes the apparent magnitude from the absolute magnitude
    Inputs:
    M: Absolute magnitude
    redshift: Redshift of the source
    '''
    # Luminosity distance:
    dL = cosmo.luminosity_distance(redshift).to(u.pc).value

    return M + 5 * np.log10(dL) - 5
r_mag = M_to_m_alt(OUT['ObsAppMagDustLine'][:, 21].flatten(), OUT['Redshift'])
i_mag = M_to_m_alt(OUT['ObsAppMagDustLine'][:, 35].flatten(), OUT['Redshift'])

In [ ]:
fig, ax = plt.subplots()

ax.hist(r_mag, np.linspace(16, 40, 100), histtype='step')
ax.hist(i_mag, np.linspace(16, 40, 100), histtype='step')

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.hist(OUT['Redshift'], 100)

ax.axvline(0.9)
ax.axvline(1.2)

plt.show()

In [ ]:
fig, ax = plt.subplots()

ax.hist(np.log10(OUT['FluxDust_OII_3729']), np.linspace(-20, -10), histtype='step')

ax.set_yscale('log')

plt.show()